# See Part two for the second question and Part Three for the third question

## Part One
Step 1: Scrape Wikipedia page

In [14]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

Step 2: Get postcodes table from Wikipedia page

In [35]:
postcodes_table = soup.find('table',{'class':'wikitable sortable'})

Step 3: Read html into a tabular format

In [36]:
import pandas as pd
df = pd.read_html(str(postcodes_table), header=0)

Step 4: Convert to Data Frame

In [37]:
df2 = pd.DataFrame(df[0])

Step 5: Rename Column

In [38]:
df2.rename(columns={'Postcode':'PostalCode'}, 
                 inplace=True)

Step 6: Replace not assigned neighbourhoods with Borough name

In [39]:
df2['Neighbourhood'].replace("Not assigned", df2['Borough'], inplace=True)

Step 7: Groupby PostalCode and combine neighbourhood names

In [40]:
df2['Neighbourhood'] = df2.groupby('PostalCode')['Neighbourhood'].transform(lambda x: "%s" % ', '.join(x)).values
df2 = df2.drop_duplicates().reset_index(drop=True)

Step 8: Drop not assigned boroughs

In [42]:
df2.drop(df2[df2['Borough'] == 'Not assigned' ].index , inplace=True)
df2.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


Step 9: Use shape method

In [43]:
df2.shape

(103, 3)

## Part Two: Geo data

Import the CSV file with geo data

In [44]:
latlon = pd.read_csv("https://cocl.us/Geospatial_data")
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join the data

In [45]:
df3 = pd.merge(df2, latlon, how='left',
        left_on='PostalCode', right_on='Postal Code', validate="1:1")
df3.drop(labels='Postal Code', axis=1, inplace=True)
df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [46]:
df3.shape

(103, 5)

## Part 3: Explore and cluster

Import libraries

In [15]:
! pip install folium

    100% |████████████████████████████████| 92kB 11.3MB/s ta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [16]:
import folium

In [17]:
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python-3.7.3               |       h33d41f4_1        36.0 MB  conda-forge
    certifi-2019.6.16          |           py37_0         148 KB  conda-forge
    pip-19.1.1                 |           py37_0         1.8 MB  conda-forge
    setuptools-41.0.1          |           py37_0         616 KB  conda-forge
    readline-8.0               |       hf8c457e_0         441 KB  conda-forge
    bzip2-1.0.6                |    h14c3975_1002         415 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    xz-5.2.4                   |    h14c3975_1001         366 KB  conda-forge
    wheel-0.33.4               |           py37_0          34 KB  conda-forge
    libffi-3.

ImportError: No module named 'distutils.sysconfig'

In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [31]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [32]:
downtown_data = df3[df3['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [33]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [34]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown